In [4]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [5]:
!pip install "dask[dataframe]"

DEPRECATION: Loading egg at /opt/anaconda3/envs/mwdc/lib/python3.12/site-packages/mwdc-1.5.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/anaconda3/envs/mwdc/lib/python3.12/site-packages/seaborn-0.13.2-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [6]:
import pandas as pd
import numpy as np
import xarray as xr
import netCDF4 as nc
from netCDF4 import Dataset

In [15]:
path2 = ('/Users/lorenzomelchionna/Repositories/multivariate-weather-data-clustering/data/ERA5_meteo_sfc_2021_daily.nc')
data = xr.open_dataset(path2, decode_times=False) #To view the date as integers of 0, 1, 2,....
data

<xarray.Dataset>
Dimensions:    (time: 365, longitude: 41, latitude: 41)
Coordinates:
  * time       (time) float64 0.0 1.0 2.0 3.0 4.0 ... 361.0 362.0 363.0 364.0
  * longitude  (longitude) float32 -33.0 -32.75 -32.5 ... -23.5 -23.25 -23.0
  * latitude   (latitude) float32 43.0 42.75 42.5 42.25 ... 33.5 33.25 33.0
Data variables:
    sp         (time, latitude, longitude) float64 ...
    sst        (time, latitude, longitude) float64 ...
    u10        (time, latitude, longitude) float64 ...
    v10        (time, latitude, longitude) float64 ...
    sshf       (time, latitude, longitude) float64 ...
    slhf       (time, latitude, longitude) float64 ...
    t2m        (time, latitude, longitude) float64 ...
Attributes:
    title:          Daily variables from ERA5 hourly data
    history:        Wed Mar 23 12:01:11 2022: ncrcat ERA5_meteo_sfc_2021_01-0...
    Input dataset:  /global/cscratch1/sd/xzheng/ENA/rawdata/ERA5_meteo_sfc_20...
    NCO:            netCDF Operators version 5.0.1 (Homepage = http://nco.sf....

In [16]:
  def detect_null(input):
    for i in input.data_vars:
      if input[i].isnull().sum().item() > 0:
        print('Missing value detected in variable', i)
    return input

In [17]:
detect_null(data)

Missing value detected in variable sst


<xarray.Dataset>
Dimensions:    (time: 365, longitude: 41, latitude: 41)
Coordinates:
  * time       (time) float64 0.0 1.0 2.0 3.0 4.0 ... 361.0 362.0 363.0 364.0
  * longitude  (longitude) float32 -33.0 -32.75 -32.5 ... -23.5 -23.25 -23.0
  * latitude   (latitude) float32 43.0 42.75 42.5 42.25 ... 33.5 33.25 33.0
Data variables:
    sp         (time, latitude, longitude) float64 1.024e+05 ... 1.011e+05
    sst        (time, latitude, longitude) float64 290.1 290.0 ... 293.4 293.3
    u10        (time, latitude, longitude) float64 -4.228 -4.348 ... 6.268 5.776
    v10        (time, latitude, longitude) float64 6.211 6.16 ... 8.584 8.798
    sshf       (time, latitude, longitude) float64 -5.477e+04 ... 2.513e+03
    slhf       (time, latitude, longitude) float64 -3.669e+05 ... -3.581e+05
    t2m        (time, latitude, longitude) float64 288.6 288.4 ... 293.1 293.1
Attributes:
    title:          Daily variables from ERA5 hourly data
    history:        Wed Mar 23 12:01:11 2022: ncrcat ERA5_meteo_sfc_2021_01-0...
    Input dataset:  /global/cscratch1/sd/xzheng/ENA/rawdata/ERA5_meteo_sfc_20...
    NCO:            netCDF Operators version 5.0.1 (Homepage = http://nco.sf....

In [18]:
def null_fill(input):

  dask_df = input.to_dask_dataframe(dim_order=None, set_index=False)
  pd_df = dask_df.compute()
  pd_df1 = pd_df.iloc[:, 3:]
  df2 = pd_df1[pd_df1.isnull().any(axis=1)]
  lst = list(df2.index.values)
  df2.loc[:] = np.nan
  dt = pd.concat([pd_df1, df2], axis=0)
  dt3 = dt[~dt.index.duplicated(keep='last')]
  dt4 = dt3[['sst', 'sp', 'u10', 'v10', 'sshf', 'slhf', 't2m']]
  pd_df4 = pd_df.iloc[:, 0:5]
  dff = pd_df4[['time', 'longitude', 'latitude', 'sst']]
  df = pd.merge(dff, dt4, left_index=True, right_index=True).drop('sst_y', axis=1)
  df.rename(columns={'sst_x':'sst'}, inplace=True)
  df_rows = pd.DataFrame(df).set_index(["time", "longitude", "latitude"])
  data = xr.Dataset.from_dataframe(df_rows)
  df_rows = pd.DataFrame(df).set_index(["time", "longitude", "latitude"])
  data = xr.Dataset.from_dataframe(df_rows)

  return data

In [19]:
data = null_fill(data)

/var/folders/gh/58gwj19d6_xdvn31s69tq1rh0000gn/T/ipykernel_32903/3653326435.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.loc[:] = np.nan


In [20]:
detect_null(data)

Missing value detected in variable sst
Missing value detected in variable sp
Missing value detected in variable u10
Missing value detected in variable v10
Missing value detected in variable sshf
Missing value detected in variable slhf
Missing value detected in variable t2m


<xarray.Dataset>
Dimensions:    (time: 365, longitude: 41, latitude: 41)
Coordinates:
  * time       (time) float64 0.0 1.0 2.0 3.0 4.0 ... 361.0 362.0 363.0 364.0
  * longitude  (longitude) float32 -33.0 -32.75 -32.5 ... -23.5 -23.25 -23.0
  * latitude   (latitude) float32 33.0 33.25 33.5 33.75 ... 42.5 42.75 43.0
Data variables:
    sst        (time, longitude, latitude) float64 294.0 293.8 ... 289.4 289.2
    sp         (time, longitude, latitude) float64 1.015e+05 ... 9.893e+04
    u10        (time, longitude, latitude) float64 -2.685 -2.525 ... 6.352 6.21
    v10        (time, longitude, latitude) float64 5.305 4.99 ... 11.29 10.92
    sshf       (time, longitude, latitude) float64 -4.078e+04 ... -2.794e+04
    slhf       (time, longitude, latitude) float64 -3.481e+05 ... -6.7e+05
    t2m        (time, longitude, latitude) float64 292.6 292.5 ... 288.7 288.7